# 네이버 OpenAPI 연동 (검색어 트렌드)

## #01. 준비과정

### [1] 패키지 참조

In [4]:
import requests
import json
import datetime as dt

### [2] 접속할 데이터의 URL

In [19]:
url = "https://openapi.naver.com/v1/datalab/search"

### [3] 요청 변수

#### 네이버에서 발급받은 인증키

In [20]:
clientId = "Gr4NLHTnI2d6E_DiD4De"
clientSecret = "Yfcz8wZJue"

#### 검색 데이터 구성

In [21]:
now = dt.datetime.now()

In [22]:
today = now.strftime("%Y-%m-%d")
today

'2023-12-15'

In [23]:
td = dt.timedelta(days=365)
td

datetime.timedelta(days=365)

In [24]:
startDateDelta = now - td
startDate = startDateDelta.strftime("%Y-%m-%d")
startDate

'2022-12-15'

In [25]:
data = {
    "startDate": startDate,
    "endDate": today,
    "timeUnit": "date",         # date: 일간, week: 주간, month: 월간
    "keywordGroups": [
        {
            "groupName": "Benz",  # 검색주제
            "keywords": ["벤츠", "e클래스", "s클래스", "c클래스", "벤츠 cls"]   # 주제와 관련된 검색어
        },
        {
            "groupName": "BMW",  # 검색주제
            "keywords": ["BMW", "5시리즈", "x5", "x7", "x3"]   # 주제와 관련된 검색어
        },
        {
            "groupName": "AUDI",  # 검색주제
            "keywords": ["아우디", "a6", "e트론", "a7", "q5"]   # 주제와 관련된 검색어
        }
    ]
}

### [4] 세션 생성

In [26]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "X-Naver-Client-Id": clientId,
    "X-Naver-Client-Secret": clientSecret
})

## #02. 데이터 요청하기

### [1] URL을 통한 데이터 요청

In [28]:
try:
    r = session.post(url, data=json.dumps(data))
    
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)

### [2] 응답 결과 확인

In [30]:
r.encoding = "utf-8"
mydict = json.loads(r.text)
print(type(mydict))
print("검색 결과 수 :", len(mydict['results']))
mydict

<class 'dict'>
검색 결과 수 : 3


{'startDate': '2022-12-15',
 'endDate': '2023-12-15',
 'timeUnit': 'date',
 'results': [{'title': 'Benz',
   'keywords': ['벤츠', 'e클래스', 's클래스', 'c클래스', '벤츠 cls'],
   'data': [{'period': '2022-12-15', 'ratio': 58.20515},
    {'period': '2022-12-16', 'ratio': 55.53207},
    {'period': '2022-12-17', 'ratio': 57.08974},
    {'period': '2022-12-18', 'ratio': 59.37145},
    {'period': '2022-12-19', 'ratio': 56.26394},
    {'period': '2022-12-20', 'ratio': 57.37935},
    {'period': '2022-12-21', 'ratio': 57.95859},
    {'period': '2022-12-22', 'ratio': 56.50268},
    {'period': '2022-12-23', 'ratio': 54.69844},
    {'period': '2022-12-24', 'ratio': 55.12113},
    {'period': '2022-12-25', 'ratio': 59.59845},
    {'period': '2022-12-26', 'ratio': 59.99373},
    {'period': '2022-12-27', 'ratio': 59.66889},
    {'period': '2022-12-28', 'ratio': 59.30883},
    {'period': '2022-12-29', 'ratio': 57.35587},
    {'period': '2022-12-30', 'ratio': 59.16011},
    {'period': '2022-12-31', 'ratio': 59.1757

## 03. 응답 데이터 활용

### [1] 데이터 재구성

```py
[
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100},
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100},
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100},
    {"날짜": "2022-12-15", "Benz": 100, "BMW": 100, "AUDI": 100}
]
```

#### 검색 결과의 `title`추출하여 데이터 구조 정의

In [34]:
struct = {'date': None}

for item in mydict['results']:
    struct[item['title']] = None
    
struct

{'date': None, 'Benz': None, 'BMW': None, 'AUDI': None}

#### 응답 결과를 탐색하면서 주어진 구조에 데이터 채워 넣기

In [45]:
mydata = []

for i, v in enumerate(mydict['results']):
    title = v['title']
    print(title)
    
    for j, w in enumerate(v['data']):
    
        # 벤츠의 데이터 수 만큼 item 추가
        if i == 0:
            current = struct.copy()
            current['date'] = w['period']
            current[title] = w['ratio']
            mydata.append(current)
        else:
            mydata[j][title] = w['ratio']
        
mydata

Benz
BMW
AUDI


[{'date': '2022-12-15', 'Benz': 58.20515, 'BMW': 71.04614, 'AUDI': 50.66337},
 {'date': '2022-12-16', 'Benz': 55.53207, 'BMW': 71.86802, 'AUDI': 45.25067},
 {'date': '2022-12-17', 'Benz': 57.08974, 'BMW': 66.89366, 'AUDI': 39.36049},
 {'date': '2022-12-18', 'Benz': 59.37145, 'BMW': 77.27681, 'AUDI': 42.0414},
 {'date': '2022-12-19', 'Benz': 56.26394, 'BMW': 71.49622, 'AUDI': 43.12942},
 {'date': '2022-12-20', 'Benz': 57.37935, 'BMW': 71.36706, 'AUDI': 46.64396},
 {'date': '2022-12-21', 'Benz': 57.95859, 'BMW': 68.16562, 'AUDI': 42.67543},
 {'date': '2022-12-22', 'Benz': 56.50268, 'BMW': 69.96986, 'AUDI': 40.63637},
 {'date': '2022-12-23', 'Benz': 54.69844, 'BMW': 69.4415, 'AUDI': 40.93381},
 {'date': '2022-12-24', 'Benz': 55.12113, 'BMW': 68.76443, 'AUDI': 38.14723},
 {'date': '2022-12-25', 'Benz': 59.59845, 'BMW': 69.91507, 'AUDI': 40.37806},
 {'date': '2022-12-26', 'Benz': 59.99373, 'BMW': 71.78192, 'AUDI': 38.84779},
 {'date': '2022-12-27', 'Benz': 59.66889, 'BMW': 72.09893, 'AUDI':